In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install tqdm


In [0]:
import cv2
import csv
from tqdm import tqdm
with tqdm(total=111753) as pbar:

  with open('/content/drive/My Drive/seen/seen-dataset/dataset_seen_training_siamese.csv', 'r') as f:
      reader = csv.reader(f)
      count=0
      print(count)
      train_data1=[]
      train_data2=[]
      train_target=[]
      for row in reader:
          if (count!=0):
              pbar.update(1)
              img1=cv2.imread('/content/drive/My Drive/seen/seen-dataset/TrainingSet/'+row[1],0)
              img2=cv2.imread('/content/drive/My Drive/seen/seen-dataset/TrainingSet/'+row[2],0)
              try: 
                img1=img1/255
                img2=img2/255
              except:
                count+=1
                continue
              train_data1.append(img1)
              train_data2.append(img2)
              train_target.append(int(row[3]))
          count=count+1   


  0%|          | 8/111753 [00:00<23:49, 78.20it/s]

0


114390it [35:12, 86.64it/s]


In [0]:
import cv2
import csv
with open('/content/drive/My Drive/seen/seen-dataset/dataset_seen_validation_siamese.csv', 'r') as f:
    reader = csv.reader(f)
    count=0
    val_data1=[]
    val_data2=[]
    val_target=[]
    for row in reader:
#         print(count)
        if (count!=0):
            img1=cv2.imread('/content/drive/My Drive/seen/seen-dataset/ValidationSet/'+row[1],0)
            img2=cv2.imread('/content/drive/My Drive/seen/seen-dataset/ValidationSet/'+row[2],0)
            try: 
              img1=img1/255
              img2=img2/255
            except:
              count+=1
              continue
            val_data1.append(img1)
            val_data2.append(img2)
            val_target.append(int(row[3]))
        count=count+1   


In [0]:
def myGenerator():
  i=0
  while i<train_data1.shape[0]:
    yield [train_data1[i:i+100], train_data2[i:i+100]], train_target[i:i+100]
    if i + 100 > train_data1.shape[0]:
      i = 0
    else:
      i += 100

In [0]:
train_data1[1:100].shape[0]

99

In [0]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
train_data1=np.array(train_data1)
train_data2=np.array(train_data2)
train_data1=np.asarray(train_data1).reshape((-1, 64, 64, 1))
train_data2=np.asarray(train_data2).reshape((-1, 64, 64, 1))
train_data1.shape

(51269, 64, 64, 1)

In [0]:
val_data1=np.array(val_data1)
val_data2=np.array(val_data2)
val_data1=np.asarray(val_data1).reshape((-1, 64, 64, 1))
val_data2=np.asarray(val_data2).reshape((-1, 64, 64, 1))

In [0]:
val_data1.shape

(906, 64, 64, 1)

In [0]:
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Activation, Flatten, Dense, Dropout
img_in = Input(shape = train_data1.shape[1:], name = 'FeatureNet_ImageInput')
n_layer = img_in
for i in range(2):
    n_layer = Conv2D(8*2**i, kernel_size = (3,3), activation = 'linear')(n_layer)
    n_layer = BatchNormalization()(n_layer)
    n_layer = Activation('relu')(n_layer)
    n_layer = Conv2D(16*2**i, kernel_size = (3,3), activation = 'linear')(n_layer)
    n_layer = BatchNormalization()(n_layer)
    n_layer = Activation('relu')(n_layer)
    n_layer = MaxPool2D((2,2))(n_layer)
n_layer = Flatten()(n_layer)
n_layer = Dense(32, activation = 'relu')(n_layer)
n_layer = Dropout(0.5)(n_layer)
n_layer = BatchNormalization()(n_layer)
n_layer = Activation('sigmoid')(n_layer)
feature_model = Model(inputs = [img_in], outputs = [n_layer], name = 'FeatureGenerationModel')
feature_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
FeatureNet_ImageInput (Input (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 62, 62, 8)         80        
_________________________________________________________________
batch_normalization_29 (Batc (None, 62, 62, 8)         32        
_________________________________________________________________
activation_29 (Activation)   (None, 62, 62, 8)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 60, 60, 16)        1168      
_________________________________________________________________
batch_normalization_30 (Batc (None, 60, 60, 16)        64        
_________________________________________________________________
activation_30 (Activation)   (None, 60, 60, 16)        0         
__________

In [0]:
from keras.layers import concatenate
img_a_in = Input(shape = train_data1.shape[1:], name = 'ImageA_Input')
img_b_in = Input(shape = train_data2.shape[1:], name = 'ImageB_Input')
img_a_feat = feature_model(img_a_in)
img_b_feat = feature_model(img_b_in)
combined_features = concatenate([img_a_feat, img_b_feat], name = 'merge_features')
combined_features = Dense(16, activation = 'sigmoid')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(4, activation = 'sigmoid')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(1, activation = 'sigmoid')(combined_features)
similarity_model = Model(inputs = [img_a_in, img_b_in], outputs = [combined_features], name = 'Similarity_Model')
similarity_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ImageA_Input (InputLayer)       (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
ImageB_Input (InputLayer)       (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
FeatureGenerationModel (Model)  (None, 32)           181712      ImageA_Input[0][0]               
                                                                 ImageB_Input[0][0]               
__________________________________________________________________________________________________
merge_features (Concatenate)    (None, 64)           0           FeatureGenerationModel[1][0]     
          

In [0]:
similarity_model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
loss_history = similarity_model.fit_generator(myGenerator(), 
                               steps_per_epoch = 500,
                               validation_data=([val_data1, val_data2], val_target),
                               epochs = 5,
                               verbose = True)

Epoch 1/5
500/500 [==============================] - 43s 86ms/step - loss: 0.6108 - acc: 0.6547 - val_loss: 1.5310 - val_acc: 0.5000
Epoch 2/5
500/500 [==============================] - 35s 70ms/step - loss: 0.5650 - acc: 0.7127 - val_loss: 0.6962 - val_acc: 0.5000
Epoch 3/5
500/500 [==============================] - 35s 70ms/step - loss: 0.5335 - acc: 0.7095 - val_loss: 0.8474 - val_acc: 0.5000
Epoch 4/5
500/500 [==============================] - 35s 70ms/step - loss: 0.5308 - acc: 0.6181 - val_loss: 0.6953 - val_acc: 0.5000
Epoch 5/5
500/500 [==============================] - 35s 71ms/step - loss: 0.5556 - acc: 0.7911 - val_loss: 0.7206 - val_acc: 0.3157


In [0]:
np.set_printoptions(threshold=np.inf)
similarity_model.predict([val_data1, val_data2])

array([[0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.3917762 ],
       [0.09030095],
       [0.40191007],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.44164822],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.39849   ],
       [0.38826668],
       [0.40608022],
       [0.40837204],
       [0.38826668],
       [0.40390107],
       [0.3950523 ],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.39121205],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.38826668],
       [0.40477857],
       [0.38826668],
       [0.388

In [0]:
import cv2
import csv
from tqdm import tqdm
with tqdm(total=100000) as pbar:

  with open('/content/drive/My Drive/shuffle/shuffled-dataset/dataset_seen_training_siamese.csv', 'r') as f:
      reader = csv.reader(f)
      count=0
      print(count)
      train_data1=[]
      train_data2=[]
      train_target=[]
      for row in reader:
          if (count!=0):
              pbar.update(1)
              img1=cv2.imread('/content/drive/My Drive/shuffle/shuffled-dataset/TrainingSet/'+row[1],0)
              img2=cv2.imread('/content/drive/My Drive/shuffle/shuffled-dataset/TrainingSet/'+row[2],0)
              try: 
                img1=img1/255
                img2=img2/255
              except:
                count+=1
                continue
              train_data1.append(img1)
              train_data2.append(img2)
              train_target.append(int(row[3]))
          count=count+1   


  0%|          | 0/100000 [00:00<?, ?it/s]

0


102278it [32:11, 113.92it/s]


In [0]:
import cv2
import csv
with open('/content/drive/My Drive/shuffle/shuffled-dataset/dataset_seen_validation_siamese.csv', 'r') as f:
    reader = csv.reader(f)
    count=0
    val_data1=[]
    val_data2=[]
    val_target=[]
    for row in reader:
#         print(count)
        if (count!=0):
            img1=cv2.imread('/content/drive/My Drive/shuffle/shuffled-dataset/ValidationSet/'+row[1],0)
            img2=cv2.imread('/content/drive/My Drive/shuffle/shuffled-dataset/ValidationSet/'+row[2],0)
            try: 
              img1=img1/255
              img2=img2/255
            except:
              count+=1
              continue
            val_data1.append(img1)
            val_data2.append(img2)
            val_target.append(int(row[3]))
        count=count+1   


In [0]:
def myGenerator():
  i=0
  while i<train_data1.shape[0]:
    yield [train_data1[i:i+100], train_data2[i:i+100]], train_target[i:i+100]
    if i + 100 > train_data1.shape[0]:
      i = 0
    else:
      i += 100

In [0]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt

In [0]:
train_data1=np.array(train_data1)
train_data2=np.array(train_data2)
train_data1=np.asarray(train_data1).reshape((-1, 64, 64, 1))
train_data2=np.asarray(train_data2).reshape((-1, 64, 64, 1))
train_data1.shape

(47168, 64, 64, 1)

In [0]:
val_data1=np.array(val_data1)
val_data2=np.array(val_data2)
val_data1=np.asarray(val_data1).reshape((-1, 64, 64, 1))
val_data2=np.asarray(val_data2).reshape((-1, 64, 64, 1))

In [0]:
val_data1.shape

(3381, 64, 64, 1)

In [0]:
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Activation, Flatten, Dense, Dropout
img_in = Input(shape = train_data1.shape[1:], name = 'FeatureNet_ImageInput')
n_layer = img_in
for i in range(2):
    n_layer = Conv2D(8*2**i, kernel_size = (3,3), activation = 'linear')(n_layer)
    n_layer = BatchNormalization()(n_layer)
    n_layer = Activation('relu')(n_layer)
    n_layer = Conv2D(16*2**i, kernel_size = (3,3), activation = 'linear')(n_layer)
    n_layer = BatchNormalization()(n_layer)
    n_layer = Activation('relu')(n_layer)
    n_layer = MaxPool2D((2,2))(n_layer)
n_layer = Flatten()(n_layer)
n_layer = Dense(32, activation = 'relu')(n_layer)
n_layer = Dropout(0.5)(n_layer)
n_layer = BatchNormalization()(n_layer)
n_layer = Activation('sigmoid')(n_layer)
feature_model = Model(inputs = [img_in], outputs = [n_layer], name = 'FeatureGenerationModel')
feature_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
FeatureNet_ImageInput (Input (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 62, 62, 8)         80        
_________________________________________________________________
batch_normalization_36 (Batc (None, 62, 62, 8)         32        
_________________________________________________________________
activation_36 (Activation)   (None, 62, 62, 8)         0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 60, 60, 16)        1168      
_________________________________________________________________
batch_normalization_37 (Batc (None, 60, 60, 16)        64        
_________________________________________________________________
activation_37 (Activation)   (None, 60, 60, 16)        0         
__________

In [0]:
from keras.layers import concatenate
img_a_in = Input(shape = train_data1.shape[1:], name = 'ImageA_Input')
img_b_in = Input(shape = train_data2.shape[1:], name = 'ImageB_Input')
img_a_feat = feature_model(img_a_in)
img_b_feat = feature_model(img_b_in)
combined_features = concatenate([img_a_feat, img_b_feat], name = 'merge_features')
combined_features = Dense(16, activation = 'sigmoid')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(4, activation = 'sigmoid')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(1, activation = 'sigmoid')(combined_features)
similarity_model = Model(inputs = [img_a_in, img_b_in], outputs = [combined_features], name = 'Similarity_Model')
similarity_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ImageA_Input (InputLayer)       (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
ImageB_Input (InputLayer)       (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
FeatureGenerationModel (Model)  (None, 32)           181712      ImageA_Input[0][0]               
                                                                 ImageB_Input[0][0]               
__________________________________________________________________________________________________
merge_features (Concatenate)    (None, 64)           0           FeatureGenerationModel[1][0]     
          

In [0]:
similarity_model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
loss_history = similarity_model.fit_generator(myGenerator(), 
                               steps_per_epoch = 500,
                               validation_data=([val_data1, val_data2], val_target),
                               epochs = 5,
                               verbose = True)

Epoch 1/5
500/500 [==============================] - 44s 87ms/step - loss: 0.5918 - acc: 0.6684 - val_loss: 1.1307 - val_acc: 0.3913
Epoch 2/5
500/500 [==============================] - 36s 71ms/step - loss: 0.6481 - acc: 0.6242 - val_loss: 1.2022 - val_acc: 0.3919
Epoch 3/5
500/500 [==============================] - 35s 71ms/step - loss: 0.6444 - acc: 0.6111 - val_loss: 0.7684 - val_acc: 0.6081
Epoch 4/5
500/500 [==============================] - 35s 71ms/step - loss: 0.6576 - acc: 0.6318 - val_loss: 0.6760 - val_acc: 0.6081
Epoch 5/5
500/500 [==============================] - 36s 71ms/step - loss: 0.6414 - acc: 0.6575 - val_loss: 0.6801 - val_acc: 0.6081


In [0]:
np.set_printoptions(threshold=np.inf)
similarity_model.predict([val_data1, val_data2])

array([[0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.46375257],
       [0.463

In [0]:
import cv2
import csv
from tqdm import tqdm
with tqdm(total=100000) as pbar:

  with open('/content/drive/My Drive/unseen/unseen-dataset/dataset_seen_training_siamese.csv', 'r') as f:
      reader = csv.reader(f)
      count=0
      print(count)
      train_data1=[]
      train_data2=[]
      train_target=[]
      for row in reader:
          if (count!=0):
              pbar.update(1)
              img1=cv2.imread('/content/drive/My Drive/unseen/unseen-dataset/TrainingSet/'+row[1],0)
              img2=cv2.imread('/content/drive/My Drive/unseen/unseen-dataset/TrainingSet/'+row[2],0)
              try: 
                img1=img1/255
                img2=img2/255
              except:
                count+=1
                continue
              train_data1.append(img1)
              train_data2.append(img2)
              train_target.append(int(row[3]))
          count=count+1   


  0%|          | 0/100000 [00:00<?, ?it/s]

0


129924it [35:41, 60.67it/s] 


In [0]:
import cv2
import csv
with open('/content/drive/My Drive/unseen/unseen-dataset/dataset_seen_validation_siamese.csv', 'r') as f:
    reader = csv.reader(f)
    count=0
    val_data1=[]
    val_data2=[]
    val_target=[]
    for row in reader:
#         print(count)
        if (count!=0):
            img1=cv2.imread('/content/drive/My Drive/unseen/unseen-dataset/ValidationSet/'+row[1],0)
            img2=cv2.imread('/content/drive/My Drive/unseen/unseen-dataset/ValidationSet/'+row[2],0)
            try: 
              img1=img1/255
              img2=img2/255
            except:
              count+=1
              continue
            val_data1.append(img1)
            val_data2.append(img2)
            val_target.append(int(row[3]))
        count=count+1   


In [0]:
def myGenerator():
  i=0
  while i<train_data1.shape[0]:
    yield [train_data1[i:i+100], train_data2[i:i+100]], train_target[i:i+100]
    if i + 100 > train_data1.shape[0]:
      i = 0
    else:
      i += 100

In [0]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt

In [0]:
train_data1=np.array(train_data1)
train_data2=np.array(train_data2)
train_data1=np.asarray(train_data1).reshape((-1, 64, 64, 1))
train_data2=np.asarray(train_data2).reshape((-1, 64, 64, 1))
train_data1.shape

(45134, 64, 64, 1)

In [0]:
val_data1=np.array(val_data1)
val_data2=np.array(val_data2)
val_data1=np.asarray(val_data1).reshape((-1, 64, 64, 1))
val_data2=np.asarray(val_data2).reshape((-1, 64, 64, 1))

In [0]:
val_data1.shape

(4316, 64, 64, 1)

In [0]:
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Activation, Flatten, Dense, Dropout
img_in = Input(shape = train_data1.shape[1:], name = 'FeatureNet_ImageInput')
n_layer = img_in
for i in range(2):
    n_layer = Conv2D(8*2**i, kernel_size = (3,3), activation = 'linear')(n_layer)
    n_layer = BatchNormalization()(n_layer)
    n_layer = Activation('relu')(n_layer)
    n_layer = Conv2D(16*2**i, kernel_size = (3,3), activation = 'linear')(n_layer)
    n_layer = BatchNormalization()(n_layer)
    n_layer = Activation('relu')(n_layer)
    n_layer = MaxPool2D((2,2))(n_layer)
n_layer = Flatten()(n_layer)
n_layer = Dense(32, activation = 'relu')(n_layer)
n_layer = Dropout(0.5)(n_layer)
n_layer = BatchNormalization()(n_layer)
n_layer = Activation('sigmoid')(n_layer)
feature_model = Model(inputs = [img_in], outputs = [n_layer], name = 'FeatureGenerationModel')
feature_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
FeatureNet_ImageInput (Input (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 62, 62, 8)         80        
_________________________________________________________________
batch_normalization_43 (Batc (None, 62, 62, 8)         32        
_________________________________________________________________
activation_43 (Activation)   (None, 62, 62, 8)         0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 60, 60, 16)        1168      
_________________________________________________________________
batch_normalization_44 (Batc (None, 60, 60, 16)        64        
_________________________________________________________________
activation_44 (Activation)   (None, 60, 60, 16)        0         
__________

In [0]:
from keras.layers import concatenate
img_a_in = Input(shape = train_data1.shape[1:], name = 'ImageA_Input')
img_b_in = Input(shape = train_data2.shape[1:], name = 'ImageB_Input')
img_a_feat = feature_model(img_a_in)
img_b_feat = feature_model(img_b_in)
combined_features = concatenate([img_a_feat, img_b_feat], name = 'merge_features')
combined_features = Dense(16, activation = 'sigmoid')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(4, activation = 'sigmoid')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(1, activation = 'sigmoid')(combined_features)
similarity_model = Model(inputs = [img_a_in, img_b_in], outputs = [combined_features], name = 'Similarity_Model')
similarity_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ImageA_Input (InputLayer)       (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
ImageB_Input (InputLayer)       (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
FeatureGenerationModel (Model)  (None, 32)           181712      ImageA_Input[0][0]               
                                                                 ImageB_Input[0][0]               
__________________________________________________________________________________________________
merge_features (Concatenate)    (None, 64)           0           FeatureGenerationModel[1][0]     
          

In [0]:
similarity_model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
loss_history = similarity_model.fit_generator(myGenerator(), 
                               steps_per_epoch = 500,
                               validation_data=([val_data1, val_data2], val_target),
                               epochs = 5,
                               verbose = True)

Epoch 1/5
500/500 [==============================] - 45s 90ms/step - loss: 0.6946 - acc: 0.5806 - val_loss: 1.7439 - val_acc: 0.4034
Epoch 2/5
500/500 [==============================] - 36s 72ms/step - loss: 0.6297 - acc: 0.7010 - val_loss: 1.2076 - val_acc: 0.4034
Epoch 3/5
500/500 [==============================] - 36s 71ms/step - loss: 0.6062 - acc: 0.7556 - val_loss: 0.8715 - val_acc: 0.4036
Epoch 4/5
500/500 [==============================] - 36s 72ms/step - loss: 0.6435 - acc: 0.6542 - val_loss: 0.6800 - val_acc: 0.5966
Epoch 5/5
500/500 [==============================] - 36s 71ms/step - loss: 0.6753 - acc: 0.6010 - val_loss: 1.7512 - val_acc: 0.5966


In [0]:
np.set_printoptions(threshold=np.inf)
similarity_model.predict([val_data1, val_data2])

array([[0.01325867],
       [0.01325727],
       [0.01326132],
       [0.01325774],
       [0.01325825],
       [0.0132587 ],
       [0.01328579],
       [0.01328534],
       [0.01328683],
       [0.01328564],
       [0.01328525],
       [0.01328516],
       [0.01328585],
       [0.01328662],
       [0.01328698],
       [0.01328665],
       [0.01328453],
       [0.01325119],
       [0.0132491 ],
       [0.01324895],
       [0.01324713],
       [0.01324841],
       [0.01324663],
       [0.01324883],
       [0.01330382],
       [0.01330274],
       [0.01330462],
       [0.01330474],
       [0.01330477],
       [0.01330495],
       [0.01329559],
       [0.01329568],
       [0.01329458],
       [0.01329422],
       [0.01329499],
       [0.01329395],
       [0.01329261],
       [0.01329422],
       [0.01329586],
       [0.01329428],
       [0.01326206],
       [0.01326585],
       [0.01326108],
       [0.01326454],
       [0.01328814],
       [0.01328757],
       [0.01328665],
       [0.013